# Path setup & import packages

In [12]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
import os
root_path = '../../../' # path to project root
sys.path.append('{}/code'.format(root_path))
sys.path.append('{}/code/core'.format(root_path))
sys.path.append('{}/code/datasets/'.format(root_path))
sys.path.insert(0,'{}/code/ptranking'.format(root_path))

from core.ranking_utils import *
from core.mallows import *
from core.ws_ranking import *
from core.ws_real_workflow import * 
from datasets.imdb_tmdb_dataset import * 
from datasets.basic_clmn_dataset import * 
from core.labelling.feature_lf import *
from ptranking_wrapper import PtrankingWrapper
import datasets_factory 
import numpy as np 
import yaml
import matplotlib.pyplot as plt
import pickle

label_feature = 'vote_average'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read config & basic setup

In [13]:
config_file_path = '{}/configs/imdb-tmdb_ranking_experiment_play.yaml'.format(root_path)

with open(config_file_path,'r') as conf_file:
    conf = yaml.full_load(conf_file)
    conf['project_root'] = root_path 

data_conf = conf['data_conf']
weak_sup_conf = conf['weak_sup_conf'] # For partial ranking experiments, we should give
l2r_training_conf = conf['l2r_training_conf']
data_conf['project_root'] = root_path

In [14]:
dataset= datasets_factory.create_dataset(data_conf)
dataset.create_samples()

Generate samples...


In [27]:
feature_list = data_conf['features']
nonbinary_features = []

df = pd.read_csv(os.path.join(root_path, 'data',
                              'imdb-tmdb', 'merged_imdb_tmdb_with_additional_features.csv')).fillna(0)
for feature in feature_list:
    if (len(df[feature].unique()) > 3) and (feature != label_feature):
        nonbinary_features.append(feature)
nonbinary_features = np.array(nonbinary_features)

In [28]:
weak_sup_conf

{'p': None,
 'checkpoint_path': 'data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs',
 'inference_rule': 'weighted_kemeny',
 'lf_features': ['runtime', 'num_voted_users', 'popularity'],
 'lf_features_highest_first_flag': [True, True, True],
 'synthetic': False,
 'recreate_if_exists': True,
 'train_method': 'triplet_opt'}

In [32]:
exp_result = {}

for trial in range(10000):
    num_lfs = np.random.randint(low=1, high=len(nonbinary_features))
    indices = np.random.choice(list(range(len(nonbinary_features))), size=num_lfs)
    wl_features = nonbinary_features[indices]
    tf = np.array([True, False])
    wl_highest_first = np.random.choice(tf, size=num_lfs)
    weak_sup_conf['lf_features'] = wl_features
    weak_sup_conf['lf_features_highest_first_flag'] = wl_highest_first
    Y_tilde, thetas = get_weak_labels(dataset, weak_sup_conf, root_path=root_path)
    r_utils = RankingUtils(data_conf['dimension'])
    kt = r_utils.mean_kt_distance(Y_tilde,dataset.Y)
    exp_result[trial] = {
        'kt': kt,
        'num_lfs': num_lfs,
        'wl_feuatres': wl_features,
        'wl_highest_first': wl_highest_first,
    }
    
    print('trial', trial, 'kt', kt)

Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 0 kt 0.4673333333333333
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 1 kt 0.5471666666666667
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2 kt 0.38216666666666665
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_m

trial 30 kt 0.49149999999999994
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 31 kt 0.38516666666666666
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 32 kt 0.4554999999999999
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 33 kt 0.5690000000000001
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl

trial 61 kt 0.39366666666666666
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 62 kt 0.46283333333333343
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 63 kt 0.5981666666666666
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 64 kt 0.507
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak

trial 92 kt 0.49583333333333335
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 93 kt 0.4820000000000001
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 94 kt 0.495
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 95 kt 0.43133333333333324
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak

trial 123 kt 0.45
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 124 kt 0.6156666666666666
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 125 kt 0.6205
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 126 kt 0.4375
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_me

...

trial 2014 kt 0.5588333333333334
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2015 kt 0.507
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2016 kt 0.5151666666666667
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2017 kt 0.49316666666666664
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use o

trial 2045 kt 0.47133333333333327
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2046 kt 0.38350000000000006
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2047 kt 0.5918333333333333
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2048 kt 0.4973333333333333
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_la

trial 2076 kt 0.435
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2077 kt 0.39233333333333337
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2078 kt 0.5271666666666666
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2079 kt 0.5135
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supe

trial 2107 kt 0.6476666666666666
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny
trial 2108 kt 0.6016666666666667
Weak labels generated and saved in ../../../data/imdb-tmdb/processed/210513_dim-10_ntrain-500_ntest-1000_model-ListMLE_weaklabel-False/LFs/weak_labels.pkl
Use our weak supervision...train_method: triplet_opt,inference_rule: weighted_kemeny


KeyboardInterrupt: 

In [34]:
kt_list = []
for trial in exp_result.keys():
    kt_list.append(exp_result[trial]['kt'])
min(kt_list)

0.3506666666666667

In [ ]:
for seed in range(5):
    
    
    if l2r_training_conf['use_weak_labels']:
        Y_tilde, thetas = get_weak_labels(dataset, weak_sup_conf, root_path=root_path)
        r_utils = RankingUtils(data_conf['dimension'])
        kt = r_utils.mean_kt_distance(Y_tilde,dataset.Y)
        print('kt distance: ', kt)
        dataset.set_Y_tilde(Y_tilde)
    else:
        kt = None
    
    ptwrapper = PtrankingWrapper(data_conf=data_conf, weak_sup_conf=weak_sup_conf,
                                 l2r_training_conf=l2r_training_conf, result_path=conf['results_path'],
                                 wl_kt_distance = kt)
    X_train, X_test, Y_train, Y_test = dataset.get_train_test_torch(use_weak_labels=l2r_training_conf['use_weak_labels'])
    ptwrapper.set_data(X_train=X_train, X_test=X_test,
                      Y_train=Y_train, Y_test=Y_test)
    model = ptwrapper.get_model()
    result = ptwrapper.train_model(model, verbose=1)
    
    with open(os.path.join(f'results/seed_{seed}.pickle'), 'wb') as f:
        pickle.dump(result, f)
        
    print(seed, max(result['test_tau']))